In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from stix2.v21 import (ThreatActor, Identity, AttackPattern, Campaign, IntrusionSet, Relationship, ExternalReference, Bundle, Grouping)
from efficient_apriori import apriori
import re
from pyattck import Attck
import requests
import json
from stix2 import MemoryStore, Filter
from neo4j import GraphDatabase
from collections import Counter
import seaborn as sns
import statistics

In [20]:
# Settings Values - for Apriori Algorithm
# TODO - 2: Do we need to adjust these numbers, in order to refine our algorithm metrics??
confidenceLevel = 0.71
supportLevel = 0.05
abstract = False

In [21]:
def NameToCode(gName):
    # create an instance of the Attck class
    attck = Attck()

    # get all APT groups in the framework
    apt_groups = attck.enterprise.actors

    # create a dictionary mapping APT group names to G codes
    group_to_gcode = {}
    for group in apt_groups:
        if group.name == gName:
            gcode = group.id
            return gcode
    return ""


In [22]:
def GenerateTestSet():
    df = pd.read_csv("datasets/Categorized_Adversary_TTPs.csv").loc[:, ['mitre_attack_name', 'mitre_attack_ttps']] # sample dataset of attacks
    test_threat_profiles = {}
    for row in df.values:
        gcode = NameToCode(row[0].strip())
        tcodes = (row[1].strip("'][").replace("'","").split(', '))
        if (len(tcodes) > 0):
            test_threat_profiles[gcode] = tcodes
    
    return test_threat_profiles
    
def GenerateAprioriLists():

    #Data from Scott at Tidal Cyber
    ttpLists = []
    df1 = json.load(open("datasets/TidalCyberData/otx_running.json")) [1:]
    for row in df1:
        if len(row[7]) > 0 and type(row[7]) == type([]):
            ttpLists.append(["'" + ttp + "'" for ttp in row[7]])
    
    df2 = json.load(open("datasets/TidalCyberData/Tidal OSINT Technique Extraction.json"))
    for row in df2:
        if len(row['Tidal Extracted Techniques']) > 0:
            ttpLists.append(["'" + ttp2 + "'" for ttp2 in row['Tidal Extracted Techniques']])
    # Some data comes from this dataset with TTPs

    # More data is gained by using attck data from tool and malware TTPs
    attack = Attck()
    
    malwares = attack.enterprise.malwares + attack.enterprise.tools
    
    for malware in malwares:
        ttpLists.append(["'" + ttp.id + "'" for ttp in malware.techniques])

    # To use the apriori we need to generate a list of lists
    aprList = ttpLists
    # for row in df3.values:
    #     aprList.append((row[13].strip('][').split(', ')))

    # get rid of empty sets
    return [ttp for ttp in aprList if len(ttp) > 1]

In [23]:
def AbstractTTPs(ttpList):
    # Take sub-techniques and remove the .### to abstract them to parent techniques 
    for i in range(0,len(ttpList)):
        ttpList[i] = [re.sub(r'\.[0-9]+', '', ttp) for ttp in ttpList[i]]
    return ttpList 

In [24]:
# takes a list of lists and returns a list of rules sorted by size 
def AprioriMining(aprList):
    # Perform apriori rule association mining
    itemsets, rules = apriori(aprList, min_support=supportLevel, min_confidence=confidenceLevel)
    
    # Sort by size to get the 1:1 mappings first and so on. 
    ruleNums = np.array([len(rule.lhs+rule.rhs) for rule in rules])
    rules = np.array(rules)
    inds = ruleNums.argsort()[::]
    rules = rules[inds]
    
    # Maximum rule size of 4 to limit number of rules, any rules with size > 4 are redundant anyways
    rules = [x for x in filter(lambda rule: len(rule.lhs+rule.rhs) <= 4, rules)]
    return rules

In [26]:
# Downloads latest MITRE framework from the branch
def get_data_from_branch(domain):
    """get the ATT&CK STIX data from MITRE/CTI. Domain should be 'enterprise-attack', 'mobile-attack' or 'ics-attack'. Branch should typically be master."""
    stix_json = requests.get(f"https://raw.githubusercontent.com/mitre-attack/attack-stix-data/master/{domain}/{domain}.json").json()
    return MemoryStore(stix_data=stix_json["objects"])

src = get_data_from_branch("enterprise-attack")

In [27]:
# Get TTPs from seed, where seed is an APT group code: GXXXX
def ExtractSeedTTPs(seed):
    seeds = []
    # This runs but says that "actors" is not found
    actor = actors[seed]
    for ttp in actor.techniques:
        seeds.append(ttp.id)
        
    # If we are using abstracted TTPs then remove sub technique
    if (abstract):
        for i in range(0, len(seeds)):
            seeds[i] = re.sub(r'\.[0-9]+', '', seeds[i])
    return seeds

In [28]:
def ExportBundle(bundle, filename):
    with open(filename, "w") as f:
        f.write(bundle.serialize())
        f.close()

In [29]:
# Each rule has a confidence, lift, support, lhs, rhs, conviction, rule power factor (RPF), 

# dictionaries to optimize requests
ttpDictionary = {}

# Takes an APTGroup and generates hypothesized relationships and attack pattern objects
# Returns a tuple of: (profile, attackPatterns, relationships, attackPatterns)
def CreateRelationships(seeds, aprioriLists):
    profile = {} #stores the TTP and tactic for easy logging
    seen = [] #stores TTPs that have been seen by the algorithm
    attackPatterns = {} #stores generated attack pattern objects
    relationships = {} # stores generated relationship objects
    # attackPatterns = {} # stores generated grouping objects

    # Get seeds out of the APT group
    # seeds = ExtractSeedTTPs(APTGroup)
    # seeds = ["T1566", "T1204"]
    
    # Perform Association Rule Mining
    rules = AprioriMining(aprioriLists) 
    
    for seed in seeds:
       ## print(seed)
        profile[seed] = 1
        ttp = 0
        # Get the stix object for the TTP
        if (not(seed in list(ttpDictionary.keys()))):
            try:          
                ttp = src.query([ Filter("external_references.external_id", "=", seed) ])[0]
                ttpDictionary[seed] = ttp
            except:
                continue
        else:
            ttp = ttpDictionary[seed]
            
        if (ttp != 0):
        # Create and add objects to lists, context is used to identify seeds from hypothesized events
            attackPatterns[seed] = ttp
    
    # print(profile)
    
    seedTotals = ["'"+x+"'" for x in seeds]
    # Use a queue to iterate through and create a tree of TTPs
    while len(seeds) > 0:
        for rule in rules:
            # Check to see if the left hand side of a rule is satisfied 
            if "'"+seeds[0]+"'" in rule.lhs and set(rule.lhs).issubset(seedTotals):
                # If the lhs is satisfied, then loop through each TTP in the rhs 
                for ttpName in rule.rhs:
                    # If this TTP hasn't been visited already then create STIX objects
                    if ttpName not in seen:
                        if (not(ttpName[1:-1] in ttpDictionary.keys())):
                            ttp = src.query([ Filter("external_references.external_id", "=", ttpName[1:-1]) ])[0]
                            ttpDictionary[ttpName[1:-1]] = ttp
                        else:
                            ttp = ttpDictionary[ttpName[1:-1]]
                        # rule tactics 
                        tactics = []
                        for i in ttp['kill_chain_phases']:
                            tactics.append(i['phase_name'])
                        
                        # create a new grouping object with the rule name. 
                        
                        attackPatterns[ttpName[1:-1]] = ttp
                        seedTotals.append(ttpName)
                        seen.append(ttpName)
                        seeds.append(ttp['external_references'][0]['external_id'])

                    # if the relationship already exists between two objects then we take the one with higher confidence
                    # make sure the exact relationship does not alraedy exist
                    if not (attackPatterns[seeds[0]]['id'], attackPatterns[ttpName[1:-1]]['id']) in relationships:
                        # if the opposite relationship exists then choose the one with the highest confidence to add
                        # we do this because if we do not then on the graph there will be two arrows going opposite directions to connect the same 2 TTPs
                        # this causes clutter and makes the confidence values unreadable since they will be layered on top of each other
                        if (attackPatterns[ttpName[1:-1]]['id'], attackPatterns[seeds[0]]['id']) in relationships:
                            # if existing relationship has higher confidence, move on
                            if float(relationships[(attackPatterns[ttpName[1:-1]]['id'], attackPatterns[seeds[0]]['id'])]['relationship_type']) > rule.confidence:
                                continue # move to next iteration of for loop, so code after this statement won't execute
                            # if existing relationship has lower confidence, delete it and let the new relationship take it's place
                            else:
                                del relationships[(attackPatterns[ttpName[1:-1]]['id'], attackPatterns[seeds[0]]['id'])]
                        relationships[(attackPatterns[seeds[0]]['id'], attackPatterns[ttpName[1:-1]]['id'])] = Relationship(attackPatterns[seeds[0]]['id'], str(round(rule.confidence, 3)), attackPatterns[ttpName[1:-1]]['id'])
                        if not((ttpName[1:-1] in profile.keys() and profile[ttpName[1:-1]] >= 1)):
                            profile[ttpName[1:-1]] = round(rule.confidence, 3)
 
        # pop to progress the queue
        seeds.pop(0) 
    
    for seed in seeds:
        profile[seed] = 1
        

    return (profile, attackPatterns, relationships)

In [30]:
aprioriList = AbstractTTPs(GenerateAprioriLists()) # generate lists for apriori
len(aprioriList)

1199

In [31]:
intelSeed = ["T1566", "T1204"] # Use-case: Observed TTPs
profile, attackPatterns, relationships = CreateRelationships(intelSeed, aprioriList)

In [32]:
# Displays the results of the Apiori Algorithm 
print(relationships)

# This is the hypothesized profile
apt_x = profile

{}


In [36]:
# Phase 2: Make APT Threat Profiles
attack = Attck()

# Dictionary to hold all the threat profiles. 
# The key is a given APT and the values are a list of TTPs
# threat_profiles = {}
# threat_profiles = json.load(open("datasets/ThreatProfiles/threat_profiles.json"))
for actor in attack.enterprise.actors:
    technique_dict = {}
    for technique in actor.techniques:
        technique_dict[technique.id[0:5]] = 1
    threat_profiles[actor.id] = technique_dict
    
# TODO: Add extended_threat profiles
# Create threatprofile Seed -- need to parse the value of the keys.
# extended_threat_profiles = {}
# extended_threat_profiles = json.load(open("datasets/ThreatProfiles/extended_threat_profiles.json"))
for apt, t_codes in threat_profiles.items():
    ttp_list = list(t_codes.keys())
    profile, attackPatterns, relationships = CreateRelationships(ttp_list, aprioriList)
    extended_threat_profiles[apt] = profile

print(extended_threat_profiles["G0025"])



NameError: name 'extended_threat_profiles' is not defined

In [ ]:
# Phase 2: WEIGHTED JACCARD SIMILARITY
# Side note: Find a way to relate techniques and sub-techniques. 

# TODO: Create a function that compares two threat profiles and produces a Weighted Jaccard. Just returns a number. 
def weighted_jac (profile1, profile2):

    # Extract the keys as sets
    keys1 = set(profile1.keys())
    keys2 = set(profile2.keys())

    # Calculate the Jaccard index
    # Only need the normal intersection and union
    intersection = keys1.intersection(keys2)
    union = keys1.union(keys2)
    # jaccard_index = len(intersection) / len(union)

    # Weight the Jaccard index by the values in the dictionaries
    weighted_intersection = sum(min(profile1[k], profile2[k]) for k in intersection)
    weighted_union = sum(max(profile1.get(k, 0), profile2.get(k, 0)) for k in union)
    
    try:
        weighted_jaccard_index = weighted_intersection / weighted_union
    except:
        print(profile1)
        print(profile2)
        return 0
        
    return weighted_jaccard_index
    #print("Weighted Jaccard index:", weighted_jaccard_index)


In [ ]:
# TODO: Create another function that calls the previous function on all of the threat profiles and our threat profile of interest.
# We could do some sort of ranking like list the APTs with the highest Weighted Jaccard first. 
def compare_apt(h_threatprofile, threat_profiles):
    # Make another dictionary 
    rankings = {}
    
    for key, value in threat_profiles.items(): 
        similarity  = (weighted_jac(value, h_threatprofile)) * 100
        percentage = round(similarity, 2)
        rankings[key] = percentage
    
    return rankings

In [ ]:
compare_apt_dict = compare_apt(apt_x, extended_threat_profiles)

# SORTED -- NICE!
sorted_dict = dict(sorted(compare_apt_dict.items(), key=lambda x: x[1], reverse=True))
print(sorted_dict)


In [ ]:
# Make some sort of visualization
import matplotlib.pyplot as plt

# Example dictionary
data = sorted_dict

# Extract keys and values from dictionary
labels = list(data.keys())
values = list(data.values())

# Create a bar chart using matplotlib
plt.bar(labels, values)

# Set chart title and axis labels
plt.title('APT THANGS')
plt.xlabel('APTs')
plt.ylabel('Similarity (%)')

# Display the chart
plt.show()


In [ ]:
# Write the dictionary to a JSON file
# import json

# with open("Extended_Threat_Profiles", "w") as outfile:
#     json.dump(extended_threat_profiles, outfile)

In [ ]:
# Generate heatmap right here
# create matrix using pandas dataset with rows and columns being APT groups

# some threat groups have no TTPs, we will filter those out

heatdf_tgroups = {}
for gcode, prof in extended_threat_profiles.items():
    if (prof != {}):
        heatdf_tgroups[gcode] = prof

aptGroups = list(heatdf_tgroups.keys())

heatdf = pd.DataFrame(columns=aptGroups)
# populate the dataframe
for gcode, prof in heatdf_tgroups.items():
    for gcode2, prof2 in heatdf_tgroups.items():
        simularity = int(weighted_jac(prof, prof2)*100)
        heatdf.loc[gcode, gcode2] = simularity

heatdf = heatdf.reset_index(drop=True).astype(int)


In [ ]:
heatdf

In [ ]:
# Convert the Pandas DataFrame to a NumPy array
arr = heatdf.to_numpy()

# Create a heatmap using the imshow() function
fig, ax = plt.subplots()
im = ax.imshow(arr, cmap='coolwarm')

# Add a colorbar to the plot
cbar = ax.figure.colorbar(im, ax=ax)

# Set the tick labels for the x and y axes
ax.set_xticks(np.arange(len(heatdf.columns)))
ax.set_yticks(np.arange(len(heatdf.index)))
ax.set_xticklabels(heatdf.columns)
ax.set_yticklabels(heatdf.index)

# Rotate the tick labels on the x axis
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# set the size of the heatmap
fig.set_size_inches(12, 12)

# Set the plot title
ax.set_title("Heatmap of heatdf")

# Show the plot
plt.show()

In [ ]:
print(threat_profiles["G0042"])

In [ ]:
## Here we evaluate our models 
test_set = GenerateTestSet() # this is just G code and associated TTPs, need to make profiles
test_profiles = {}
#test_profiles = json.load(open("datasets/ThreatProfiles/test_profiles.json"))
# create profiles using the test data and make it into a dictionary
# we filter data here to remove all empty lists
i = 0
for gcode, tcodes in (test_set.items()):
    i += 1
    if (tcodes == ['']):
        continue
    tcodes = [tc.split('.')[0] if '.' in tc else tc for tc in tcodes]
    profile, attackPatterns, relationships = CreateRelationships(tcodes, aprioriList)
    
    test_profiles[gcode] = profile


len(list(test_profiles.keys()))

In [ ]:
# topn: we consider the function a success if the atp group is identified in the top n results.
def evaluate_model(topn):
    res = [] # 1 for hit and 0 for no hit
    conf = []
    hits = 0
    total = 0
    topn = 1
    
    
    for gcode, tcodes in test_profiles.items():
            
        rankings = Counter(compare_apt(tcodes, extended_threat_profiles)).most_common(topn)
        total += 1
        lasthit = hits
        for k,v in rankings:
            if (gcode == k):
                conf.append(v)
                hits += 1
                break
        if (lasthit == hits):
            res.append(0)
        else:
            res.append(1)
    
    print(statistics.mean(conf), statistics.stdev(conf))
    sd = statistics.stdev(res)
    return((hits/total, sd))

In [ ]:
evaluate_model(1)

In [ ]:
# exporting data
with open("datasets/ThreatProfiles/threat_profiles.json", "w") as outfile:
    json.dump(threat_profiles, outfile)

with open("datasets/ThreatProfiles/extended_threat_profiles.json", "w") as outfile:
    json.dump(extended_threat_profiles, outfile)

with open("datasets/ThreatProfiles/test_profiles.json", "w") as outfile:
    json.dump(test_profiles, outfile)

In [ ]:
def json_query (extended_threat_profiles): 
    apt_list = []
    ttp_list = {}
    relationship_list = []
    for g_code, t_code_list in extended_threat_profiles.items(): 
        # Get APTs based on its G-code
        apt = src.query([Filter("external_references.external_id", "=", g_code)])[0]
        apt_list.append(apt)
        # Get TTPs based on its T-code
        for t_code, confidence in t_code_list.items():
            if (not(t_code in ttp_list.keys())):
                ttp = src.query([Filter("external_references.external_id", "=", t_code)])[0]
                ttp_list[t_code] = ttp
            conf = float(confidence)*100
            conf = str(conf) + '%'
            if (confidence > 0.85):
                #Create relationships between every TTP and a given APT
                relationship = Relationship(apt, conf, ttp_list[t_code])
                relationship_list.append(relationship)
    
    return (apt_list, list(ttp_list.values()), relationship_list)
    



In [ ]:
# Save the lists
apt_sdo_list, ttp_sdo_list, relationship_sdo_list = json_query(extended_threat_profiles)

In [ ]:
json_query(extended_threat_profiles)

# Print statements to verify that the query worked
print(apt_sdo_list[0])
print(ttp_sdo_list[0])
print(relationship_sdo_list[0])

In [ ]:
# Global Database Driver Variable
uri = "neo4j+s://ae3c581e.databases.neo4j.io"
user = "neo4j"
password = "NIpuTQWR1jR_1EWwpCvZ-eu873ADJznwfrON5WiMwik"

driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
def keyCheck(dictionary, key):
        if key in dictionary:
            return dictionary[key]
        else:
            return None

In [ ]:
def addAPT(apt_sdo_list):
    with driver.session(database="neo4j") as session:
        for elem in apt_sdo_list:
            query = (
                "CREATE (p1:APT { spec_version: $spec_version, id: $id, created_by_ref: $created_by_ref, created: $created, modified: $modified, name: $name, description: $description, aliases: $aliases, external_references: $external_references}) "
                "RETURN p1"
            )
            result = session.run(query, spec_version = str(elem['spec_version']), id = str(elem['id']), created_by_ref = str(keyCheck(elem, 'created_by_ref')), created = str(keyCheck(elem, 'created')), modified = str(keyCheck(elem, 'modified')), name = str(keyCheck(elem, 'name')), description = str(keyCheck(elem, 'description')), aliases = str(keyCheck(elem, 'aliases')), external_references = str(keyCheck(elem, 'external_references')))

In [ ]:
def addTTP(ttp_sdo_list):
        with driver.session(database="neo4j") as session:
            for elem in ttp_sdo_list:
                query = (
                    "CREATE (p1:TTP { spec_version: $spec_version, id: $id, created_by_ref: $created_by_ref, created: $created, modified: $modified, name: $name, kill_chain_phases: $kill_chain_phases, external_references: $external_references, x_mitre_is_subtechnique: $x_mitre_is_subtechnique}) "
                    "RETURN p1"
                )
                result = session.run(query, spec_version = str(elem['spec_version']), id = str(elem['id']), created_by_ref = str(keyCheck(elem, 'created_by_ref')), created = str(keyCheck(elem, 'created')), modified = str(keyCheck(elem, 'modified')), name = str(keyCheck(elem, 'name')), kill_chain_phases = str(keyCheck(elem, 'kill_chain_phases')), external_references = str(keyCheck(elem, 'external_references')), x_mitre_is_subtechnique = bool(elem['x_mitre_is_subtechnique']))

In [ ]:
# Need to filter the relationships a little better -- I probably just need to query something in the GUI version
# Probably could add something about the realtionship, tbh

def addRelationship(relationship_sdo_list):
    with driver.session(database="neo4j") as session:
            for elem in relationship_sdo_list:
                rel_type = elem['relationship_type']
                
                query = (
                    "MATCH (a1: APT) "
                    "WHERE a1.id = $sourceId "
                    "MATCH (aP1: TTP) "
                    "WHERE aP1.id = $targetId "
                    "CREATE (a1)-[: USES { id: $id, confidence_level: $relationship_type, description: $description, source_ref: $source_ref, target_ref: $target_ref}]->(aP1) "
                    "RETURN a1, aP1"
                )

                
                result = session.run(query, sourceId = elem['source_ref'], targetId = elem['target_ref'], spec_version = str(keyCheck(elem, 'spec_version')), id = str(elem['id']), created_by_ref = str(keyCheck(elem, 'created_by_ref')), created = str(keyCheck(elem, 'created')), modified = str(keyCheck(elem, 'modified')), relationship_type = str(keyCheck(elem, 'relationship_type')), description = keyCheck(elem, 'description'), source_ref = str(keyCheck(elem, 'source_ref')), target_ref = str(keyCheck(elem, 'target_ref')))


In [ ]:

def buildDatabase(apt_sdo_list, ttp_sdo_list, relationship_sdo_list):
    # Add the SDO nodes to the database
    addAPT(apt_sdo_list)
    addTTP(ttp_sdo_list)
    addRelationship(relationship_sdo_list)
    driver.close()
    

In [ ]:

buildDatabase(apt_sdo_list, ttp_sdo_list, relationship_sdo_list)